In [1]:
import pandas as pd

In [2]:
CATEGORY_GROUPS_IN_QUESTION = {
  'Animal Control': ['Pick up Dead Animal', 'Animal Generic Request'],
  'Abandoned Vehicles': ['Abandoned Vehicles', 'Abandoned Bicycle'],
  'Rodent Removal': ['Rodent Activity', 'Bed Bugs', 'Mice Infestation - Residential'],
  'Living Conditions': ['Unsatisfactory Living Conditions', 'Poor Conditions of Property', 'Unsanitary Conditions - Establishment', 'Illegal Occupancy', 'Heat - ,Excessive  Insufficient'],
  'Graffiti Removal': ['Graffiti Removal']
}

In [ ]:
def make_q1_json():
  df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
  df = df_orig[['OPEN_DT', 'TYPE', 'tract_and_block_group']]
  d = make_top_n_dict(df)
  d2 = make_top_n_dict_for_all_yrs(df)
  d3 = incorporate_totals_into_orig_dict(d, d2)

  with open('static/top_5_types_by_yr_loc.json', 'w') as data_file:    
    json.dump(d3, data_file, ensure_ascii=False)  

In [3]:
df_orig = pd.read_pickle('../data/data_from_remove_from_dataset.pkl')
df_orig.shape

(516406, 40)

In [38]:
categs = []

In [39]:
for i in CATEGORY_GROUPS_IN_QUESTION.values():
    categs.extend(i)

In [62]:
mapping = {}

In [64]:
for k,v in CATEGORY_GROUPS_IN_QUESTION.items():
    for vi in v:
        mapping[vi] = k

In [41]:
df = df_orig[['tract_and_block_group', 'TYPE', 'COMPLETION_HOURS_LOG_10']][df.TYPE.isin(categs)]
df.shape

(64621, 3)

In [65]:
print df.TYPE.drop_duplicates().shape
df.TYPE = df.TYPE.map(mapping)
print df.TYPE.drop_duplicates().shape

905400                      Pick up Dead Animal
905607              Poor Conditions of Property
904994                       Abandoned Vehicles
905119         Unsatisfactory Living Conditions
903689                   Animal Generic Request
905128    Unsanitary Conditions - Establishment
902506                        Abandoned Bicycle
903294                          Rodent Activity
900172                         Graffiti Removal
899432                                 Bed Bugs
788307           Mice Infestation - Residential
295262                        Illegal Occupancy
Name: TYPE, dtype: object
905400        Animal Control
905607     Living Conditions
904994    Abandoned Vehicles
903294        Rodent Removal
900172      Graffiti Removal
Name: TYPE, dtype: object


In [48]:
from collections import defaultdict

In [72]:
d_means = defaultdict(defaultdict)
d_std_devs = defaultdict(defaultdict)

In [66]:
df_agg = df.groupby(['tract_and_block_group', 'TYPE'])

In [76]:
for row in df_agg.mean().reset_index().to_dict('records'):
    d_means[row['tract_and_block_group']][row['TYPE']] = row['COMPLETION_HOURS_LOG_10']

In [78]:
for row in df_agg.std().reset_index().to_dict('records'):
    d_std_devs[row['tract_and_block_group']][row['TYPE']] = row['COMPLETION_HOURS_LOG_10']

In [83]:
import json

In [91]:
with open('static/q2_means.json', 'w') as data_file:    
    json.dump(d_means, data_file, ensure_ascii=False)  

In [90]:
with open('static/q2_std.json', 'w') as data_file:    
    json.dump(d_std_devs, data_file, ensure_ascii=False)  

In [88]:
d_means['0001001']

defaultdict(None,
            {'Abandoned Vehicles': 2.422750802073733,
             'Animal Control': 0.5638010470133301,
             'Graffiti Removal': 2.4607065758100086,
             'Living Conditions': 1.9169505595884735,
             'Rodent Removal': 2.314285802448486})

In [87]:
d_std_devs['0001001']

defaultdict(None,
            {'Abandoned Vehicles': 0.4517375494245284,
             'Animal Control': 0.7157933310362354,
             'Graffiti Removal': 0.6382321252011757,
             'Living Conditions': 1.2788714718272272,
             'Rodent Removal': 0.8374164159124523})